In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
import  matplotlib.pyplot as plt
import joblib
from sklearn.metrics import mean_absolute_percentage_error
import random
import torch
from torch import  nn,optim
import os
from torch.utils import data

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed=521)


data = pd.read_csv('data.csv')


X = data.iloc[:, :-1]
Y = data.iloc[:, -1]


class EI:
    def __init__(self,trainin,testin):
        train=self.trainin
        test=self.testin
        adi=self.ad(adr=0.2,trains=train,threshold=0.03)
        trainad=train.drop(index=adi).reset_index(drop=True)
        train,test=self.ts(trainad,test)
    def ad(adr,trains,sdel):
        ei=[]
        train=trains
        index=[i for i in train.columns][1:-1]
        threshold_num=sdel*len(trains)
        for ind in index:
            top,low=train.loc[:, ind].max(),train.loc[:, ind].min()
            diff=top-low
            diffmin,diffmax=low+adr*diff,low+(1-adr)*diff
            minlist,maxlist=train[train[inx] < diffmin].index.values.tolist(),train[train[inx] > diffmax].index.values.tolist()
            if len(minlist) <= threshold_num:
                for minindex in train[train[inx] < diffmin].index.values.tolist():
                    ei.append(minindex)
            if len(maxlist) <= threshold_num:
                for maxindex in train[train[inx] > diffmax].index.values.tolist():
                    ei.append(maxindex)
        ei=list(set(ei))
        ei=sorted(ei)
        return ei
    def ts(trains,tests):
        train=trains
        test=tests
        data = pd.concat([train, test], axis=0)

        roll_cols = train.columns
        for i in [j for j in range(1, 6)] + [12, 24, 48, 60]:
            data[[ii+f'_mean_{i}' for ii in roll_cols]] = data[roll_cols].rolling(i, min_periods=1).mean()

        pd.set_option("display.max_columns", 100)
        save_rate = 0.5
        data = data[data.describe().T.sort_values(by="std", ascending=False, inplace=False).iloc[: int(len(data.columns)*save_rate)].index.tolist()]

        traint = data.iloc[:train.shape[0]].reset_index(drop=True)
        testt = data.iloc[train.shape[0]:].reset_index(drop=True)
        return traint,testt
    def adts(self):
        return train,test
ei=EI()
train,test=EI.adts()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class ATN(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim):
        super(ATN, self).__init__()

		#定义层
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.sig1 = nn.Sigmoid() #nn.Linear为线性关系，加上激活函数转为非线性
        self.drop1=nn.Dropout(p=0.7)


        self.fc2 = nn.Linear(hidden_dim, output_dim)


    def forward(self, x):
        out = self.fc1(x)
        out = self.sig1(out)
        out = self.drop1(out)
        out = self.fc2(out)
        return out
input_dim = X.shape[1]  #一维向量长度
hidden_dim = 16            #hidden layer 神经元个数
output_dim = Y.shape[1]  #1个类
batch_size=32
modelts = ATN(input_dim, hidden_dim, output_dim).to(device)
myloss = nn.MSELoss()     #均方差
optimizer = optim.Adam(modelts.parameters(),lr=0.001)
